In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 보스턴 주택 가격 데이터 세트를 Peceptron 기반에서 학습 및 테스트하기 위한 데이터 로드
 * 사이킷런에서 보스턴 주택 가격 데이터 세트를 로드하고 이를 DataFrame으로 생성

In [2]:
from sklearn.datasets import load_boston

boston = load_boston()
print(boston)

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]]), 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 1

In [3]:
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target #target값
print(bostonDF.shape)
bostonDF.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Weight와 Bias의 Update 값을 계산하는 함수 생성.
* w1은 RM(방의 계수) 피처의 Weight 값
* w2는 LSTAT(하위계층 비율) 피처의 Weight 값
* bias는 Bias
* N은 입력 데이터 건수
![](https://raw.githubusercontent.com/chulminkw/CNN_PG/main/utils/images/Weight_update.png)


In [4]:
# gradient_descent()함수에서 반복적으로 호출되면서 update될 weight/bias 값을 계산하는 함수. 
# rm은 RM(방 개수), lstat(하위계층 비율), target은 PRICE임. 전체 array가 다 입력됨. 
# 반환 값은 weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 loss로 반환.
'''
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01):
    # 데이터 건수
    N = len(target)
    # 예측 값. 
    predicted = w1 * rm + w2*lstat + bias
    # 실제값과 예측값의 차이 
    diff = target - predicted
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산. (위 내용 참고, 미분 취하는 계산임)
    w1_update = -(2/N)*learning_rate*(np.dot(rm.T, diff))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat.T, diff))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff))
    
    # Mean Squared Error값을 계산. 
    mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 반환. 
    return bias_update, w1_update, w2_update, mse_loss
'''
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate):
    # rm, lstat의 영향이 가장 커서 이 두 개로만 연습할 것임
    # w1, w2가 들어오는 이유는 예측값 계산하기 위함
    
    N = len(target) # def의 인자로 들어오는 건 전부 array(배열) => sum 속도빠름. loop돌리지 않고 numpy 써서 더 좋음.
    # 주택가격 전체 건수
    
    predicted = w1*rm + w2*lstat + bias
    # 예측값i
    
    diff = target-predicted # array 인수 506개를 한번에 계산
    # 차이i = 실제값i-예측값i
    
    w1_update = -learning_rate*(2/N)*np.dot(rm.T, diff) # np.dot()위해 앞에있는 rm 전치행렬로 바꿈
    w2_update = -learning_rate*(2/N)*np.dot(lstat.T, diff)
    # η(dLoss(W)/dw1)
    # η = learning rate = step
    
    bias_update1 = -learning_rate*(2/N)*np.sum(diff)
    bias_factors = np.ones((N,))
    bias_update = -learning_rate*(2/N)*np.dot(bias_factors.T, diff)
    bias_update1
    bias_update
    
    mse_loss = np.mean(np.square(diff)) # 차이의 제곱의 평균
    
    return bias_update, w1_update, w2_update, mse_loss # 업데이트할 bias와 weight, mse 리턴

### Gradient Descent 를 적용하는 함수 생성
* iter_epochs 수만큼 반복적으로 get_update_weights_value()를 호출하여 update될 weight/bias값을 구한 뒤 Weight/Bias를 Update적용. 

In [5]:
np.zeros((3,)), np.ones((2,))

(array([0., 0., 0.]), array([1., 1.]))

In [6]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.ones((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0] # 현재 features 안에는 bostonDF에서 rm, lstat 두 피처만 뽑혀있었음
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행 (get_update_weights_value 함수) 
    for i in range(iter_epochs):
        # weight/bias update 값 계산 
        bias_update, w1_update, w2_update, loss = get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate)
        # weight/bias의 update 적용 (이전 셀 참고)
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', loss) #loss값이 계속 줄어드나 확인용
        
    return w1, w2, bias

### Gradient Descent 적용
* 신경망은 데이터를 정규화/표준화 작업을 미리 선행해 주어야 함. 
* 이를 위해 사이킷런의 MinMaxScaler를 이용하여 개별 feature값은 0~1사이 값으로 변환후 학습 적용.

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() # default값 안 주면 0~1 사이 값으로 바꿔줌
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']]) #bostonDF에서 rm, lstat 2개의 피처만 가져올것임
print(scaled_features[:15])
print(bostonDF['PRICE'])
print(bostonDF['PRICE'].values)

[[0.57750527 0.08967991]
 [0.5479977  0.2044702 ]
 [0.6943859  0.06346578]
 [0.65855528 0.03338852]
 [0.68710481 0.09933775]
 [0.54972217 0.09602649]
 [0.4696302  0.29525386]
 [0.50028741 0.48068433]
 [0.39662771 0.7781457 ]
 [0.46809734 0.424117  ]
 [0.53956697 0.51655629]
 [0.46905537 0.31843267]
 [0.44606246 0.38576159]
 [0.45755892 0.18018764]
 [0.48572523 0.23537528]]
0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: PRICE, Length: 506, dtype: float64
[24.  21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 15.  18.9 21.7 20.4
 18.2 19.9 23.1 17.5 20.2 18.2 13.6 19.6 15.2 14.5 15.6 13.9 16.6 14.8
 18.4 21.  12.7 14.5 13.2 13.1 13.5 18.9 20.  21.  24.7 30.8 34.9 26.6
 25.3 24.7 21.2 19.3 20.  16.6 14.4 19.4 19.7 20.5 25.  23.4 18.9 35.4
 24.7 31.6 23.3 19.6 18.7 16.  22.2 25.  33.  23.5 19.4 22.  17.4 20.9
 24.2 21.7 22.8 23.4 24.1 21.4 20.  20.8 21.2 20.3 28.  23.9 24.8 22.9
 23.9 26.6 22.5 22.2 23.6

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() # default값 안 주면 0~1 사이 값으로 바꿔줌
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']]) #bostonDF에서 rm, lstat 2개의 피처만 가져올것임

w1, w2, bias = gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, verbose=True)
# bostonDF['PRICE'].values를 해줘서 value값만 들어있는 array배열이 됨
# iter_epochs=1000 => 1000번돌리는것. 
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)
'''
최초 w1, w2, bias: [0.] [0.] [1.]
Epoch: 1 / 1000
w1: [0.24193162] w2: [0.10311943] bias: [1.43065613] loss: 548.0813043478261
Epoch: 2 / 1000
w1: [0.47767212] w2: [0.20269304] bias: [1.84955238] loss: 522.964778344195
Epoch: 3 / 1000
w1: [0.70739021] w2: [0.29881838] bias: [2.25700994] loss: 499.19625820107575
~~~
Epoch: 998 / 1000
w1: [18.53484608] w2: [-12.98953596] bias: [16.7777374] loss: 38.49782159620822
Epoch: 999 / 1000
w1: [18.54107799] w2: [-13.00014391] bias: [16.77768681] loss: 38.48266313399792
Epoch: 1000 / 1000
w1: [18.54730416] w2: [-13.01074165] bias: [16.77763613] loss: 38.46753354662152

##### 최종 w1, w2, bias #######
[18.54730416] [-13.01074165] [16.77763613]
'''

최초 w1, w2, bias: [0.] [0.] [1.]
Epoch: 1 / 1000
w1: [0.24193162] w2: [0.10311943] bias: [1.43065613] loss: 548.0813043478261
Epoch: 2 / 1000
w1: [0.47767212] w2: [0.20269304] bias: [1.84955238] loss: 522.964778344195
Epoch: 3 / 1000
w1: [0.70739021] w2: [0.29881838] bias: [2.25700994] loss: 499.19625820107575
Epoch: 4 / 1000
w1: [0.93124998] w2: [0.39159032] bias: [2.65334123] loss: 476.7031232605375
Epoch: 5 / 1000
w1: [1.14941104] w2: [0.48110116] bias: [3.03885015] loss: 455.41666565492966
Epoch: 6 / 1000
w1: [1.36202867] w2: [0.56744065] bias: [3.41383227] loss: 435.2718794853261
Epoch: 7 / 1000
w1: [1.56925388] w2: [0.65069613] bias: [3.7785751] loss: 416.20726135905875
Epoch: 8 / 1000
w1: [1.77123356] w2: [0.73095251] bias: [4.13335831] loss: 398.1646216743121
Epoch: 9 / 1000
w1: [1.96811059] w2: [0.8082924] bias: [4.47845392] loss: 381.0889060727257
Epoch: 10 / 1000
w1: [2.16002396] w2: [0.88279616] bias: [4.81412652] loss: 364.9280265121499
Epoch: 11 / 1000
w1: [2.34710885] w2:

'\n최초 w1, w2, bias: [0.] [0.] [1.]\nEpoch: 1 / 1000\nw1: [0.24193162] w2: [0.10311943] bias: [1.43065613] loss: 548.0813043478261\nEpoch: 2 / 1000\nw1: [0.47767212] w2: [0.20269304] bias: [1.84955238] loss: 522.964778344195\nEpoch: 3 / 1000\nw1: [0.70739021] w2: [0.29881838] bias: [2.25700994] loss: 499.19625820107575\n~~~\nEpoch: 1000 / 1000\nw1: [18.54730416] w2: [-13.01074165] bias: [16.77763613] loss: 38.46753354662152\n##### 최종 w1, w2, bias #######\n[18.54730416] [-13.01074165] [16.77763613]\n'

### 계산된 Weight와 Bias를 이용하여 Price 예측
* 예측 feature 역시 0~1사이의 scaled값을 이용하고 Weight와 bias를 적용하여 예측값 계산. 

In [10]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
print(predicted)
# 506개의 피쳐 * 18.547 + 506개의 피쳐 * (-13.01) + bias
# 즉 506개의 예측값이 나옴. 

[26.32199985 24.28120724 28.83088566 28.55765177 28.22912024 25.72412455
 21.64653851 19.80255933 14.00975824 19.94150312 20.06436833 21.33430362
 20.03184795 22.91974577 22.72412289 22.43566492 23.47313948 20.76415887
 19.93629525 21.04657277 16.99180918 20.97688724 19.8503221  18.2646619
 19.94441163 18.71403189 20.08487328 20.02969382 23.23019    24.16071412
 16.93275783 21.64076697 15.94043741 18.41593039 19.10524111 22.35308212
 21.40503436 22.38482097 22.30880043 26.63002653 28.99470609 27.06525392
 24.58117202 24.14523564 22.88307157 21.27079875 20.22587115 19.42357475
 12.86934154 18.83598828 21.10620573 23.08962849 25.98685157 23.03284945
 20.35499875 28.77829662 25.35604676 27.5482532  24.11891387 22.49690667
 20.42496602 20.76144126 25.27081906 25.36380869 27.099783   25.42046063
 21.63317771 22.72486824 19.92409776 22.5020238  25.13581086 22.38078542
 24.31569104 24.23016338 24.60252209 23.87324688 22.76055162 22.87690012
 22.46068688 22.35163698 26.75088312 25.67417128 24.

In [9]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
# scaled_features : rm, lstat. 학습을 0~1로 scaled된 값으로 했기 때문에, 예측도 0~1 scaled된 값으로 해야 함. 
bostonDF['PREDICTED_PRICE'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.322000
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.281207
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.830886
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.557652
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.229120
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.724125
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.646539
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.802559
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.009758
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.941503


### Keras를 이용하여 보스턴 주택가격 모델 학습 및 예측
* Dense Layer를 이용하여 퍼셉트론 구현. units는 1로 설정. 

**Keras 기초 설명**

* 우리가 앞에서 만들었던 코드는 모두 keras의 Dense layer 하나로 해결할 수 있음. (keras의 대표적인 기본 layer)
* 우리가 keras에서 해야 할 것
    1. 입력이 어떻게 생겨 있는가
    2. 커널 초기화 / bias 초기화

In [13]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [14]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # deep learning의 심층 신경망은 Sequential을 활용해 이렇게 편하게 만들 수 있음. 

    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음. 
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용. 
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
'''
    unit의 개수 => 1개라서 single layer. 근데 node 자체도 1개 뿐임. 
        *참고. unit개수 3개면 세로로 3개 가진 layer 만들어짐
    입력값 => 2개의 컬럼(rm, lstat) 가진 1차원 데이터. 
        *참고. Dense에서 입력값 정해주는 건 '첫 번째 layer'에서만 가능!
    kernel_initializer => weight값 초기화. 아까 weight는 0으로 초기화해줬기 때문에 'zeros'
    bias_initializer => bias값 초기화. 아까 bias는 1로 초기화해줬기 때문에 'ones'
    
    
    ★하나짜리 Dense layer 하나 만들어짐. 그 안에는 rm, lstat, 1개의 node 들어있음(?더알아보기?)
    ★위에서 다 코드로 해줬지만, 이것처럼 Dense를 사용하면 한 번에 해줄 수 있음. 
'''

# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행. 
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model.fit(scaled_features, bostonDF['PRICE'].values, epochs=1000)

'''
Epoch 1/1000
16/16 [==============================] - 1s 2ms/step - loss: 542.3250 - mse: 542.3250
Epoch 2/1000
16/16 [==============================] - 0s 2ms/step - loss: 530.1854 - mse: 530.1854
Epoch 3/1000
16/16 [==============================] - 0s 2ms/step - loss: 518.1102 - mse: 518.1102
~
Epoch 998/1000
16/16 [==============================] - 0s 2ms/step - loss: 30.5184 - mse: 30.5184
Epoch 999/1000
16/16 [==============================] - 0s 2ms/step - loss: 30.5257 - mse: 30.5257
Epoch 1000/1000
16/16 [==============================] - 0s 2ms/step - loss: 30.5215 - mse: 30.5215

아까 위에서 keras 없이 numpy를 써서 직접 계산한 최소 loss는 38.46이었다. 
지금 keras를 쓴 결과를 보면 똑같이 mse로 계산했지만 loss가 30.52로 더 줄어든 모습을 볼 수 있다. 
얘가 조금 더 정밀하게 계산됐음!
'''

2022-01-08 00:38:54.293016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 00:38:54.385647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 00:38:54.386433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-08 00:38:54.387642: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/1000
16/16 [==============================] - 1s 2ms/step - loss: 542.3250 - mse: 542.3250
Epoch 2/1000
16/16 [==============================] - 0s 2ms/step - loss: 530.1854 - mse: 530.1854
Epoch 3/1000
16/16 [==============================] - 0s 2ms/step - loss: 518.1102 - mse: 518.1102
Epoch 4/1000
16/16 [==============================] - 0s 2ms/step - loss: 506.5363 - mse: 506.5363
Epoch 5/1000
16/16 [==============================] - 0s 2ms/step - loss: 494.9872 - mse: 494.9872
Epoch 6/1000
16/16 [==============================] - 0s 2ms/step - loss: 483.7516 - mse: 483.7516
Epoch 7/1000
16/16 [==============================] - 0s 2ms/step - loss: 472.8481 - mse: 472.8481
Epoch 8/1000
16/16 [==============================] - 0s 2ms/step - loss: 462.0984 - mse: 462.0984
Epoch 9/1000
16/16 [==============================] - 0s 2ms/step - loss: 451.5417 - mse: 451.5417
Epoch 10/1000
16/16 [==============================] - 0s 2ms/step - loss: 441.3442 - mse: 441.3442
Epoch 11/

### Keras로 학습된 모델을 이용하여 주택 가격 예측 수행. 

In [15]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE'] = predicted
bostonDF.head(10)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE,KERAS_PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.322000,28.962353
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.281207,25.488424
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.830886,32.618553
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.557652,32.396496
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.229120,31.583378
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.724125,28.090538
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.646539,21.310619
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.802559,17.734711
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.009758,8.025681
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.941503,18.231962


### Stochastic Gradient Descent와 Mini Batch Gradient Descent 구현
* SGD 는 전체 데이터에서 한건만 임의로 선택하여 Gradient Descent 로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용
* Mini Batch GD는 전체 데이터에서 Batch 건수만큼 데이터를 선택하여 Gradient Descent로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
print(bostonDF.shape)
bostonDF.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### SGD 기반으로 Weight/Bias update 값 구하기

In [2]:
def get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate=0.01):
    
    # 데이터 건수
    N = target_sgd.shape[0] # 1건만 가져옴. N=1
    # 예측 값. 
    predicted_sgd = w1 * rm_sgd + w2*lstat_sgd + bias
    # 실제값과 예측값의 차이 
    diff_sgd = target_sgd - predicted_sgd
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_sgd.T, diff_sgd))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_sgd.T, diff_sgd))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_sgd))
    
    # Mean Squared Error값을 계산. 
    #mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값 반환 
    return bias_update, w1_update, w2_update

### SGD 수행하기

In [3]:
print(bostonDF['PRICE'].values.shape)
print(np.random.choice(bostonDF['PRICE'].values.shape[0], 1))
print(np.random.choice(506, 1))

(506,)
[447]
[15]


In [10]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def st_gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # ★★  iteration 시마다 stochastic gradient descent 를 수행할 데이터를 한개만 추출. 추출할 데이터의 인덱스를 random.choice() 로 선택. 
        # ★★  이 부분이 차이점!!
        stochastic_index = np.random.choice(target.shape[0], 1) # target.shape는 (506, ), [0]로 506만 뽑음. choice로 0~505 中 인덱스 하나 가져옴.
        rm_sgd = rm[stochastic_index]
        lstat_sgd = lstat[stochastic_index]
        target_sgd = target[stochastic_index]
        # SGD 기반으로 Weight/Bias의 Update를 구함.  
        bias_update, w1_update, w2_update = get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate)
        
        # SGD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            # ★★  Loss는 전체 학습 데이터 기반으로 구해야 함. 그래서 다시 계산. 
            predicted = w1 * rm + w2*lstat + bias # 506크기
            diff = target - predicted # 506크기
            mse_loss = np.mean(np.square(diff)) # 506크기
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

w1, w2, bias = st_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

'''
# iter_epochs = 1000 일 경우

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 1000
w1: [0.19886952] w2: [0.15447682] bias: [0.428] loss: 566.4890310332081
Epoch: 2 / 1000
w1: [0.3328932] w2: [0.2414122] bias: [0.70656217] loss: 550.1790711145755
Epoch: 3 / 1000
w1: [0.47761565] w2: [0.33858328] bias: [1.05835838] loss: 530.6927173377886~
~
Epoch: 998 / 1000
w1: [17.96850385] w2: [-12.50058998] bias: [17.82107867] loss: 40.18492577801926
Epoch: 999 / 1000
w1: [17.88330738] w2: [-12.56324186] bias: [17.65214236] loss: 39.808483480402664
Epoch: 1000 / 1000
w1: [17.85103876] w2: [-12.57553742] bias: [17.59034055] loss: 39.713398541363546
##### 최종 w1, w2, bias #######
[17.85103876] [-12.57553742] [17.59034055]

--------------------------------------------------------
loss가 그냥 gd(38.xx)와 sgd(39.xx) 사이에 큰 차이가 없음. 
그러면 속도 빠른 sgd 쓰는 게 더 좋은가?
--------------------------------------------------------

# iter_epochs = 5000 일 경우

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 5000
w1: [0.17649358] w2: [0.08399007] bias: [0.38] loss: 569.898885290007
Epoch: 2 / 5000
w1: [0.64095319] w2: [0.22834435] bias: [1.0455724] loss: 528.4635402071557
Epoch: 3 / 5000
w1: [0.87788183] w2: [0.30449688] bias: [1.47477426] loss: 504.31500590685295
~
Epoch: 4998 / 5000
w1: [24.97696842] w2: [-22.28155063] bias: [16.52014672] loss: 30.74411093054529
Epoch: 4999 / 5000
w1: [24.9192096] w2: [-22.32033865] bias: [16.41437715] loss: 30.6757660974208
Epoch: 5000 / 5000
w1: [24.88880826] w2: [-22.34915887] bias: [16.35290306] loss: 30.654980848238058
##### 최종 w1, w2, bias #######
[24.88880826] [-22.34915887] [16.35290306]

--------------------------------------------------------
loss가 줄어드는 방향성으로 계속 가긴 하지만 epoch 뒤로 갈 수록 별 차이 없음. 
그리고 30 초반 구간에서 34, 35, 31, 32 .... 이렇게 수렴하는 부분이 지그재그임. 
놀라운 점은 sgd에서는 단 1 건으로만 gd를 했는데도 불구하고 iteration을 반복하면서 이렇게 수렴해서 최저점을 찾아준다는 점. 
--------------------------------------------------------
'''

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 5000
w1: [0.17649358] w2: [0.08399007] bias: [0.38] loss: 569.898885290007
Epoch: 2 / 5000
w1: [0.64095319] w2: [0.22834435] bias: [1.0455724] loss: 528.4635402071557
Epoch: 3 / 5000
w1: [0.87788183] w2: [0.30449688] bias: [1.47477426] loss: 504.31500590685295
Epoch: 4 / 5000
w1: [0.97708116] w2: [0.42638753] bias: [1.70026263] loss: 491.6913569051716
Epoch: 5 / 5000
w1: [1.09255685] w2: [0.51130619] bias: [1.92923971] loss: 479.10576750430874
Epoch: 6 / 5000
w1: [1.13987518] w2: [0.65125048] bias: [2.11325936] loss: 469.4878820346937
Epoch: 7 / 5000
w1: [1.74707888] w2: [0.77339477] bias: [2.91371303] loss: 424.81282380285893
Epoch: 8 / 5000
w1: [1.88189256] w2: [0.89761106] bias: [3.23050393] loss: 409.41097808223066
Epoch: 9 / 5000
w1: [2.0569844] w2: [0.93945614] bias: [3.58731937] loss: 392.80695525370675
Epoch: 10 / 5000
w1: [2.20048532] w2: [1.04388057] bias: [3.90479766] loss: 378.1160910402112
Epoch: 11 / 5000
w1: [2.27088014] w2: [1.

'\n# iter_epochs = 1000 일 경우\n\n최초 w1, w2, bias: [0.] [0.] [0.]\nEpoch: 1 / 1000\nw1: [0.19886952] w2: [0.15447682] bias: [0.428] loss: 566.4890310332081\nEpoch: 2 / 1000\nw1: [0.3328932] w2: [0.2414122] bias: [0.70656217] loss: 550.1790711145755\nEpoch: 3 / 1000\nw1: [0.47761565] w2: [0.33858328] bias: [1.05835838] loss: 530.6927173377886~\n~\nEpoch: 998 / 1000\nw1: [17.96850385] w2: [-12.50058998] bias: [17.82107867] loss: 40.18492577801926\nEpoch: 999 / 1000\nw1: [17.88330738] w2: [-12.56324186] bias: [17.65214236] loss: 39.808483480402664\nEpoch: 1000 / 1000\nw1: [17.85103876] w2: [-12.57553742] bias: [17.59034055] loss: 39.713398541363546\n##### 최종 w1, w2, bias #######\n[17.85103876] [-12.57553742] [17.59034055]\n\n\nloss가 그냥 gd(38.xx)와 sgd(39.xx) 사이에 큰 차이가 없음. \n그러면 속도 빠른 sgd 쓰는 게 더 좋은가?\n\n\n# iter_epochs = 5000 일 경우\n\n'

In [13]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_SGD'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_SGD
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.722050
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.422176
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.216934
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,31.997354
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.234008
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,27.888721
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.442764
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,18.061570
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,8.833592
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,18.524630


### iteration시마다 일정한 batch 크기만큼의 데이터를 random하게 가져와서 GD를 수행하는 Mini-Batch GD 수행

In [14]:
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    
    # 데이터 건수
    N = target_batch.shape[0]  # sgd에서는 N=1이었지만, 미니배치에서는 batch 사이즈만큼 들어옴. 
    # 예측 값. 
    predicted_batch = w1 * rm_batch+ w2 * lstat_batch + bias
    # 실제값과 예측값의 차이 
    diff_batch = target_batch - predicted_batch
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))
    
    # Mean Squared Error값을 계산. 
    #mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값 반환 
    return bias_update, w1_update, w2_update

In [15]:
batch_indexes = np.random.choice(506, 30)
print(batch_indexes) # 506개의 인덱스 값 中 랜덤으로 뽑은 30개의 인덱스
'''
[438  68 221 395 181 155   3 142 159 233 224 479 323 333 257 371 175 247
 178 443 173 145 416 298 389 364 118 313 480 467]
'''

bostonDF['RM'].values[batch_indexes] # 그 인덱스에 해당하는 30개의 값들
'''
array([5.935, 5.594, 6.164, 6.471, 6.144, 6.152, 6.998, 5.403, 6.51 ,
       8.247, 8.266, 6.229, 5.708, 6.316, 8.704, 6.216, 6.546, 6.226,
       6.86 , 6.485, 6.416, 6.13 , 6.782, 6.345, 5.39 , 8.78 , 5.872,
       6.266, 6.242, 6.003])
'''


[438  68 221 395 181 155   3 142 159 233 224 479 323 333 257 371 175 247
 178 443 173 145 416 298 389 364 118 313 480 467]


array([5.935, 5.594, 6.164, 6.471, 6.144, 6.152, 6.998, 5.403, 6.51 ,
       8.247, 8.266, 6.229, 5.708, 6.316, 8.704, 6.216, 6.546, 6.226,
       6.86 , 6.485, 6.416, 6.13 , 6.782, 6.345, 5.39 , 8.78 , 5.872,
       6.266, 6.242, 6.003])

In [16]:
# ★★ batch_gradient_descent()만의 차이점 : 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_random_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # batch_size 갯수만큼 데이터를 임의로 선택. 
        batch_indexes = np.random.choice(target.shape[0], batch_size)
        rm_batch = rm[batch_indexes]
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        # Batch GD 기반으로 Weight/Bias의 Update를 구함. 
        bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
        
        # Batch GD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            # Loss는 전체 학습 데이터 기반으로 구해야 함.
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [19]:
w1, w2, bias = batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)
'''
iter_epochs = 1000 일 경우

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 1000
w1: [0.22943614] w2: [0.11344454] bias: [0.44693333] loss: 565.3387557934267
Epoch: 2 / 1000
w1: [0.43129521] w2: [0.23717088] bias: [0.84768168] loss: 541.7839255975184
Epoch: 3 / 1000
w1: [0.64893603] w2: [0.34521567] bias: [1.24924687] loss: 518.5714356906904
~
Epoch: 998 / 1000
w1: [18.57204136] w2: [-12.74136525] bias: [16.77546822] loss: 38.73329542551714
Epoch: 999 / 1000
w1: [18.59705407] w2: [-12.75449555] bias: [16.79012423] loss: 38.70840897287523
Epoch: 1000 / 1000
w1: [18.60780287] w2: [-12.76230497] bias: [16.79129104] loss: 38.69429777666282
##### 최종 w1, w2, bias #######
[18.60780287] [-12.76230497] [16.79129104]

-----------------------------------------------------
loss는 38.69
-----------------------------------------------------

iter_epochs = 5000 일 경우

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 5000
w1: [0.21950095] w2: [0.12660859] bias: [0.41853333] loss: 566.6909400464812
Epoch: 2 / 5000
w1: [0.46855542] w2: [0.23282252] bias: [0.87970859] loss: 539.5653582799514
Epoch: 3 / 5000
w1: [0.76126972] w2: [0.32961609] bias: [1.37010349] loss: 511.0838349412638
~
Epoch: 4998 / 5000
w1: [25.75016694] w2: [-23.43397964] bias: [16.30173302] loss: 30.542635992699037
Epoch: 4999 / 5000
w1: [25.77254205] w2: [-23.43184258] bias: [16.33755707] loss: 30.558286190914306
Epoch: 5000 / 5000
w1: [25.76737047] w2: [-23.43189828] bias: [16.33575656] loss: 30.55669856847968
##### 최종 w1, w2, bias #######
[25.76737047] [-23.43189828] [16.33575656]

-----------------------------------------------------
loss는 30.55
loss가 줄어드는 숫자를 살펴보면, sgd보다 mini-batch에서 비교적 수렴이 지그재그가 덜함. 

'''

최초 w1, w2, bias: [0.] [0.] [0.]
Epoch: 1 / 5000
w1: [0.21950095] w2: [0.12660859] bias: [0.41853333] loss: 566.6909400464812
Epoch: 2 / 5000
w1: [0.46855542] w2: [0.23282252] bias: [0.87970859] loss: 539.5653582799514
Epoch: 3 / 5000
w1: [0.76126972] w2: [0.32961609] bias: [1.37010349] loss: 511.0838349412638
Epoch: 4 / 5000
w1: [0.97383755] w2: [0.43555746] bias: [1.74428915] loss: 489.90196532479837
Epoch: 5 / 5000
w1: [1.16178039] w2: [0.5495884] bias: [2.12058481] loss: 469.6452146100008
Epoch: 6 / 5000
w1: [1.34847413] w2: [0.64014875] bias: [2.46581424] loss: 451.332148415123
Epoch: 7 / 5000
w1: [1.62294422] w2: [0.71899527] bias: [2.90470461] loss: 428.21667584378554
Epoch: 8 / 5000
w1: [1.83639097] w2: [0.80035127] bias: [3.28044853] loss: 409.33982128180816
Epoch: 9 / 5000
w1: [2.02614835] w2: [0.88941856] bias: [3.6296392] loss: 392.33288517800196
Epoch: 10 / 5000
w1: [2.2502746] w2: [0.96867881] bias: [3.98217018] loss: 375.06824832199004
Epoch: 11 / 5000
w1: [2.43432618] w2

'\niter_epochs = 1000 일 경우\n\n최초 w1, w2, bias: [0.] [0.] [0.]\nEpoch: 1 / 1000\nw1: [0.22943614] w2: [0.11344454] bias: [0.44693333] loss: 565.3387557934267\nEpoch: 2 / 1000\nw1: [0.43129521] w2: [0.23717088] bias: [0.84768168] loss: 541.7839255975184\nEpoch: 3 / 1000\nw1: [0.64893603] w2: [0.34521567] bias: [1.24924687] loss: 518.5714356906904\n~\nEpoch: 998 / 1000\nw1: [18.57204136] w2: [-12.74136525] bias: [16.77546822] loss: 38.73329542551714\nEpoch: 999 / 1000\nw1: [18.59705407] w2: [-12.75449555] bias: [16.79012423] loss: 38.70840897287523\nEpoch: 1000 / 1000\nw1: [18.60780287] w2: [-12.76230497] bias: [16.79129104] loss: 38.69429777666282\n##### 최종 w1, w2, bias #######\n[18.60780287] [-12.76230497] [16.79129104]\n\n-----------------------------------------------------\n\niter_epochs = 5000 일 경우\n\n\n-----------------------------------------------------\n\n비교적 \n\n'

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH_RANDOM'] = predicted
bostonDF.head(10)

### iteration 시에 순차적으로 일정한 batch 크기만큼의 데이터를 전체 학습데이터에 걸쳐서 가져오는 Mini-Batch GD 수행

In [ ]:
for batch_step in range(0, 506, 30):
    print(batch_step)

In [ ]:
bostonDF['PRICE'].values[480:510]

In [ ]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # batch_size 만큼 데이터를 가져와서 weight/bias update를 수행하는 로직을 전체 데이터 건수만큼 반복
        for batch_step in range(0, target.shape[0], batch_size):
            # batch_size만큼 순차적인 데이터를 가져옴. 
            rm_batch = rm[batch_step:batch_step + batch_size]
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]
        
            bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)

            # Batch GD로 구한 weight/bias의 update 적용. 
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
        
            if verbose:
                print('Epoch:', i+1,'/', iter_epochs, 'batch step:', batch_step)
                # Loss는 전체 학습 데이터 기반으로 구해야 함.
                predicted = w1 * rm + w2*lstat + bias
                diff = target - predicted
                mse_loss = np.mean(np.square(diff))
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [ ]:
w1, w2, bias = batch_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)

### Mini BATCH GD를 Keras로 수행
* Keras는 기본적으로 Mini Batch GD를 수행

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음. 
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용. 
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행. 
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])

# Keras는 반드시 Batch GD를 적용함. batch_size가 None이면 32를 할당. 
model.fit(scaled_features, bostonDF['PRICE'].values, batch_size=30, epochs=1000)

In [ ]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)